In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import os
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

In [2]:
## 读取数据
train_data = pd.read_csv('train_nn.csv', sep=' ')
test_data = pd.read_csv('test_nn.csv', sep=' ')

In [3]:
X = train_data.columns
feature_cols = [col for col in X if col not in ['price','SaleID']]
## 提前特征列，标签列构造训练样本和测试样本
X_train = train_data[feature_cols]
X_test = test_data[feature_cols]

In [4]:
x = np.array(X_train)
y = np.array(train_data['price'])
x_test = np.array(X_test)

In [6]:
logdir = os.path.join("callbacks")
output_model_file = os.path.join(logdir,"model.h5")  # 输出的model文件
callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file, save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]

In [16]:
kfolder = KFold(n_splits=5, shuffle=True, random_state=7)
zero = np.zeros(len(x))
predictions = np.zeros(len(x_test))
predictions_train = np.zeros(len(x))
kfold = kfolder.split(x, y)

for train_inx,valid_inx in kfold:
    x_train = x[train_inx]
    y_train = y[train_inx]
    x_valid = x[valid_inx]
    y_valid = y[valid_inx]

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)))
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)))
    model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)))
    model.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.02)))
    model.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(),metrics=['mae'])
    model.fit(x_train,y_train,batch_size =512,epochs=500,validation_data=(x_valid, y_valid), callbacks=callbacks)
    zero[valid_inx] = model.predict(x_valid).reshape((model.predict(x_valid).shape[0],))
    predictions += model.predict(x_test).reshape((model.predict(x_test).shape[0],)) / kfolder.n_splits
    predictions_train += model.predict(x).reshape((model.predict(x).shape[0],)) / kfolder.n_splits

Epoch 1/500
235/235 [==============================] - 3s 14ms/step - loss: 3480.5056 - mae: 3462.7534 - val_loss: 1377.3254 - val_mae: 1342.5182
Epoch 2/500
235/235 [==============================] - 3s 12ms/step - loss: 1211.7534 - mae: 1171.1053 - val_loss: 1109.3164 - val_mae: 1065.7935
Epoch 3/500
235/235 [==============================] - 3s 13ms/step - loss: 1053.0853 - mae: 1008.1841 - val_loss: 989.4257 - val_mae: 943.0027
Epoch 4/500
235/235 [==============================] - 3s 12ms/step - loss: 959.3339 - mae: 912.2347 - val_loss: 916.9647 - val_mae: 869.6070
Epoch 5/500
235/235 [==============================] - 3s 14ms/step - loss: 899.9656 - mae: 852.2472 - val_loss: 881.4904 - val_mae: 833.6614
Epoch 6/500
235/235 [==============================] - 3s 14ms/step - loss: 858.2123 - mae: 810.2964 - val_loss: 833.7025 - val_mae: 786.0568
Epoch 7/500
235/235 [==============================] - 3s 13ms/step - loss: 821.3643 - mae: 773.6117 - val_loss: 827.4697 - val_mae: 779.4

In [17]:
print("mae: {:<8.8f}".format(mean_absolute_error(zero, y)))

mae: 546.63205492


In [18]:
# 测试集输出
predictions[predictions < 0] = 0
res = pd.DataFrame()
res['SaleID'] = test_data.SaleID
res['price'] = predictions
res.head()

,SaleID,price
0,200000,1221.347366
1,200001,1905.472748
2,200002,8623.842407
3,200003,1245.840134
4,200004,1947.860931
